### Summary
Note book này được sao chép và chỉnh sửa lại từ project của [Nick Kuzmenkov](https://www.kaggle.com/nickuzmenkov).

Việc trùng lặp ảnh gây khó khăn cho việc huấn luyện dữ liệu. Ảnh trùng lặp nhưng cùng nhãn gây hiện tương data leakage, trong khi ảnh trùng lặp nhưng khác nhãn gây nhiễu.
Note book này sử dụng để tìm cách ảnh trùng lặp. Kết quả tìm được 56 cặp ảnh trùng lặp.
### Các note book khác của nhóm:

1. [Revealing Duplicates notebook](https://www.kaggle.com/nvlinhh/int3414-22-n11-revealing-duplicate)
2. [Preprocessing notebook](https://www.kaggle.com/congnguyen8201/int3414-22-n11-preprocessing)
3. [Training notebook](https://www.kaggle.com/congnguyen8201/int3414-22-n11-training)

### Imports

In [ ]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imagehash
import PIL
import os

## 1. Saving downscaled images to boost performance
Computing hash over original images of very high quality would take nearly 5 hours, thus we downscaling first.

In [ ]:
class CFG():
    
    threshold = .9
    img_size = 600
    seed = 42

In [ ]:
#đọc ảnh vào
root = '/kaggle/input/plant-pathology-2021-fgvc8/train_images'
#lấy path của file jpg
paths = os.listdir(root)

df = pd.read_csv('/kaggle/input/plant-pathology-2021-fgvc8/train.csv', index_col='image')

#decode ảnh, chuyển ảnh sang dạng 600X600X3 (rgb), cast sang unit8
for path in tqdm(paths, total=len(paths)):
    image = tf.io.read_file(os.path.join(root, path))
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [CFG.img_size, CFG.img_size])
    image = tf.cast(image, tf.uint8).numpy()
    plt.imsave(path, image)

## 2. Hash computation

In [ ]:
hash_functions = [#grey scale, down scale sang 8x8
    imagehash.average_hash,#tính trung bình cộng tất cả các điểm ảnh rồi so sánh từng điểm ảnh, >tb ->1, <tb -> 0
    imagehash.phash,#
    imagehash.dhash,
    imagehash.whash]
#mỗi hàm hash đưa ra tập 64 bit 0,1

image_ids = []
hashes = []#kết quả sau khi áp dụng 4 cách hash cho ảnh.

paths = tf.io.gfile.glob('./*.jpg')

for path in tqdm(paths, total=len(paths)):

    image = PIL.Image.open(path)

    hashes.append(np.array([x(image).hash for x in hash_functions]).reshape(-1,))
    image_ids.append(path.split('/')[-1])#thêm id ảnh
    
hashes = np.array(hashes)
image_ids = np.array(image_ids)

## 3. Run search across hashed images
We firstly compare each image hash with all the hashes and then leave only unique pairs of matches

In [ ]:
#so sánh các ảnh
duplicate_ids = []
# ảnh đầu: 110 ->ttf
#so sanh ảnh a2: 100>tff
#similarity: 101 -> mean(similarity) = 2/3
for i in tqdm(range(len(hashes)), total=len(hashes)):
    similarity = (hashes[i] == hashes).mean(axis=1)#mảng độ tương đồng của các điểm ảnh
    duplicate_ids.append(list(image_ids[similarity > CFG.threshold]))#nếu độ giống nhau > 0.9 thì coi là giống nhau
duplicate_ids

In [ ]:
duplicates = [frozenset([x] + y) for x, y in zip(image_ids, duplicate_ids)]
duplicates = set([x for x in duplicates if len(x) > 1])#do sau khi so sánh duplicate_id có 1 phần tử là chính ảnh đó
len(duplicates)

Here we add some of the duplicates spotted by @kingofarmy in the corresponding **[discussion](https://www.kaggle.com/c/plant-pathology-2021-fgvc8/discussion/229851)**:

## 4. Let's see what is found

In [ ]:
#cho ra một file csv để dùng cho note book sau
print(f'Found {len(duplicates)} duplicate pairs:')
for row in duplicates:
    print(', '.join(row))

In [ ]:
print('Writing duplicates to "duplicates.csv".')
with open('duplicates.csv', 'w') as file:
    for row in duplicates:
        file.write(','.join(row) + '\n')

In [ ]:
for row in duplicates:
    
    figure, axes = plt.subplots(1, len(row), figsize=[5 * len(row), 5])

    for i, image_id in enumerate(row):
        image = plt.imread(os.path.join('../input/plant-pathology-2021-fgvc8/train_images', image_id))
        axes[i].imshow(image)

        axes[i].set_title(f'{image_id} - {df.loc[image_id, "labels"]}')
        axes[i].axis('off')

    plt.show()

### Clear working folder to avoid output pollution

In [ ]:
for file in tf.io.gfile.glob('./*.jpg'):
    os.remove(file)

### Acknowledgements

* This work is Copy&Edit form @appian **[notebook](https://www.kaggle.com/appian/let-s-find-out-duplicate-images-with-imagehash)** with a lot of changes, but still highly inspired. If you find this notebook useful, please, upvote his work too.
* Thanks to @kingofarmy for spotting more duplicates in **[his thread](https://www.kaggle.com/c/plant-pathology-2021-fgvc8/discussion/229851)**.